In [16]:
from __future__ import print_function, division
import gensim 
import re
import multiprocessing
import os
import numpy as np
import nltk
from collections import Counter
from time import time
import codecs

In [2]:
print(gensim.__version__)

0.12.3


In [3]:
gensim.models.word2vec.FAST_VERSION

1

In [21]:
DATA = "/home/marat/ck12_clean.txt"
WORD2VEC_EXPORT = "/home/marat/data/word2vec.txt"

In [5]:
!wc -l /home/marat/ck12.txt

272578 /home/marat/ck12.txt


In [6]:
def parse_text(t):
    s = re.sub(r'[^\w\s-]', '', t)
    r = s.lower().split()
    return r

In [7]:
def iter_lines(fname):
    with open(fname) as f:
        for l in f:
            r = parse_text(l)
            if len(r) > 0:
                yield r

In [8]:
for i, t in enumerate(iter_lines(DATA)):
    print(t)
    if i > 1:
        break

['scientifically', 'there', 'is', 'an', 'actual', 'definition', 'of', 'living', 'living', 'organisms', 'must', 'have', 'certain', 'characteristics', 'if', 'they', 'do', 'not', 'have', 'these', 'characteristics', 'are', 'they', 'living', 'this', 'butterfly', 'like', 'all', 'other', 'insects', 'animals', 'plants', 'and', 'every', 'other', 'living', 'organism', 'shares', 'common', 'characteristics', 'with', 'all', 'life', 'what', 'exactly', 'does', 'it', 'mean', 'to', 'be', 'alive', 'this', 'chapter', 'will', 'answer', 'this', 'question', 'these', 'concepts', 'will', 'serve', 'as', 'an', 'introduction', 'to', 'life', 'science', 'discussing', 'the', 'basics', 'of', 'studying', 'the', 'life', 'sciences', 'and', 'addressing', 'the', 'question', 'what', 'is', 'a', 'living', 'organism']
['if', 'you', 'let', 'your', 'emotions', 'rule', 'your', 'thinking', 'you', 'might', 'think', 'a', 'thumping', 'noise', 'was', 'a', 'ghost', 'if', 'you', 'think', 'like', 'a', 'scientist', 'however', 'you', 'as

In [9]:
class SentencesStopWords(object):    
    def __init__(self, filename):
        self.filename = filename
        
    def __iter__(self):
        with codecs.open(self.filename, encoding='utf8') as f:
            for line in (line.strip() for line in f):
                words = parse_text(line)
                yield u' '.join(words)

In [269]:
from nltk.stem.snowball import SnowballStemmer
import codecs

# class SentencesStopWords1(object):    
#     def __init__(self, filename):
#         self.filename = filename
#         self.sent_tokenize = sent_tokenize
#         self.word_tokenize = nltk.tokenize.TreebankWordTokenizer()
#         self.stop = set(nltk.corpus.stopwords.words('english'))
#         c = Counter()
#         with codecs.open('/home/marat/ck12_clean.txt', encoding='utf8') as f:
#             for words in (line.strip().split() for line in f):
#                 for w in words:
#                     c[wn.morphy(w)] += 1
        
#     def __iter__(self):
#         with codecs.open(self.filename, encoding='utf8') as f:
#             for line in (line.strip() for line in f):
#                 words = [w for w in parse_text(line) if w not in self.stop]
#                 yield u' '.join(words)
                
class SentencesMorphy(object):    
    def __init__(self, filename):
        self.filename = filename
        self.stop = set(nltk.corpus.stopwords.words('english'))
        self.stemmer = SnowballStemmer("english")
        
    def proc_line(self, line):
        words = []
        for w in parse_text(line):
            if w in self.stop:
                continue
            b = wn.morphy(w)
            words.append(self.stemmer.stem(b if b else w))
        return words
        
    def __iter__(self):
        with codecs.open(self.filename, encoding='utf8') as f:
            for line in (line.strip() for line in f):
                words = self.proc_line(line)
                yield u' '.join(words)

In [270]:
%%time
sentences = SentencesMorphy('/home/marat/ck12_clean.txt')
with codecs.open('/home/marat/ck12_morphy.txt', mode='w', encoding='utf8') as f:
    for s in sentences:
        print(s, file=f)

7435
CPU times: user 42.9 s, sys: 172 ms, total: 43.1 s
Wall time: 43 s


In [271]:
from nltk.corpus import wordnet as wn
from collections import Counter
c = Counter()
sentences = SentencesMorphy('/home/marat/ck12_clean.txt')
for s in sentences:
    for w in s.split():
        c[w] += 1
print(len([w for w in c if c[w]==1]))

for w in c:
    if c[w] == 1:
        print(w, wn.synsets(w), sep='\t')

xylen	[]
monoatom	[]
bornth	[]
2553	[]
non-yeast	[]
scum-to	[]
francesco	[]
rictor	[]
flagar	[]
over-s	[]
darron	[]
usercharmed88wikipedia	[]
scola	[]
re-emerg	[]
mssin	[]
benitez	[]
semitranspar	[]
4ni	[]
basic-	[]
medicinethat	[]
delf	[Synset('delf.n.01')]
deli	[Synset('delicatessen.n.02')]
966	[]
sars-lik	[]
pudgi	[]
microbi	[]
lebaron	[]
mulitmedia	[]
chanut	[]
vrublevski	[]
-700	[]
florey	[Synset('florey.n.01')]
cancer-promot	[]
andra	[]
-sublevel	[]
he-n	[]
esoy	[]
budget	[Synset('budget.n.01'), Synset('budget.n.02'), Synset('budget.v.01')]
sooth	[Synset('sooth.n.01')]
275	[]
276	[]
targa	[]
di-substitut	[]
1850-1936	[]
sponsler	[]
hackemann	[]
succumb	[Synset('yield.v.13'), Synset('succumb.v.02')]
cyp2c18	[]
guillemin	[]
hero	[Synset('hero.n.01'), Synset('hero.n.02'), Synset('champion.n.02'), Synset('hero.n.04'), Synset('hero.n.05'), Synset('hero.n.06'), Synset('bomber.n.03')]
hoblitt	[]
coulombssecond	[]
terriberri	[]
energysuch	[]
he-3	[]
usertimswikipedia	[]
1670s	[]
chaotic	

In [10]:
%%time

# sentences = gensim.models.word2vec.LineSentence("/home/marat/ck12_clean.txt")  # 36.12
# sentences = gensim.models.word2vec.LineSentence("/home/marat/ck12.txt")  # 34.3
# sentences = gensim.models.word2vec.LineSentence("/home/marat/ck12_token.txt")  # 35.68
# sentences = gensim.models.word2vec.LineSentence("/home/marat/ck12_stop.txt")  # 36.5
# sentences = gensim.models.word2vec.LineSentence("/home/marat/ck12_punkt.txt")  # 36.04
# sentences = gensim.models.word2vec.LineSentence("/home/marat/ck12_stop1.txt")  # 36.68 !
# sentences = gensim.models.word2vec.LineSentence("/home/marat/ck12_morphy.txt")  # 36.8
sentences = gensim.models.word2vec.LineSentence("/home/marat/ck12_tokens.txt") 
# word_model = gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=300, iter=5, window=20)
word_model = gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=100, iter=20, window=20)
# models = [gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=300, iter=15, window=5), 
# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=300, iter=20, window=5),
# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=300, iter=25, window=5),

# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=150, iter=20, window=5),
# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=250, iter=20, window=5),
# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=350, iter=20, window=5),
# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=450, iter=20, window=5),

# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=300, iter=20, window=5),
# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=300, iter=20, window=15),
# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=300, iter=20, window=25),
# gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count()-1, size=300, iter=20, window=35)]

# ss = SentencesMorphy('/home/marat/ck12_clean.txt')
# def quantify_text(txt):
#     ws = ss.proc_line(txt)
#     return [w for w in ws if w in word_model.vocab]


# def range_answers(q, answers):
#     question = quantify_text(q)
#     ps = Counter()
#     scores = []
#     for a in answers:
#         a_q = quantify_text(a)
#         if a_q != []:
#             si = word_model.n_similarity(question, a_q)
#             scores.append(si)
#         else:
#             scores.append(0)
#     return scores

# for e in range(20):
#     start_time = time()
#     word_model.train(sentences)
    
#     TRAIN_SET = '/home/marat/Downloads/training_set.tsv'
#     tries = []
#     with open(TRAIN_SET) as f:
#         next(f)
#         for i, l in enumerate(f):
#             [qid, q, r, aa, ab, ac, ad] = l.strip().split("\t")
#             scores = range_answers(q, [aa, ab, ac, ad])
#             guess = "ABCD"[scores.index(max(scores))]
#             tries.append(1 if guess == r else 0)
#         print(np.mean(tries), '%.0fsec' % (time() - start_time))

CPU times: user 12min, sys: 5.44 s, total: 12min 5s
Wall time: 4min 15s


In [19]:
word_model.most_similar("sun")

[(u'planets', 0.6914310455322266),
 (u'orbit', 0.6760215759277344),
 (u'moon', 0.6752529740333557),
 (u'planet', 0.6730597019195557),
 (u'earth', 0.652760922908783),
 (u'venus', 0.6225054264068604),
 (u'orbits', 0.6038405299186707),
 (u'\xd710', 0.5924381613731384),
 (u'neptune', 0.5861884355545044),
 (u'sun\u2019s', 0.5755904912948608)]

In [17]:
def quantify_text(txt):
    words = []
    for s in nltk.sent_tokenize(txt):
        for w in nltk.word_tokenize(s.lower()):
            words.append(w)
    return [w for w in words if w in word_model.vocab if len(w) > 2]

def range_answers(q, answers):
    question = quantify_text(q)
    ps = Counter()
    scores = []
    for a in answers:
        a_q = quantify_text(a)
        if a_q != []:
            si = word_model.n_similarity(question, a_q)
            scores.append(si)
        else:
            scores.append(0)
    return scores

TRAIN_SET = '/home/marat/Downloads/training_set.tsv'
TRAIN_SET_ANSWERS = '/home/marat/allen-training-answers.tsv'
tries = []
ranks = []
scores_correct = []
scores_choice = []
scores_diff = []
with codecs.open(TRAIN_SET, encoding='utf8') as f:
    with codecs.open(TRAIN_SET_ANSWERS, 'w', encoding='utf8') as fo:
        next(f)
        for i, l in enumerate(f):
            [qid, q, r, aa, ab, ac, ad] = l.strip().split("\t")
            scores = np.array(range_answers(q, [aa, ab, ac, ad]))
            
            score_of_correct = scores['ABCD'.index(r)]
            score_of_choice = max(scores)
            guess = "ABCD"[np.argmin(score_of_choice)]
            tries.append(1 if guess == r else 0)
            ranks.append(sorted(scores, reverse=True).index(score_of_correct))
            scores_choice.append(score_of_choice)
            scores_correct.append(score_of_correct)
            scores_diff.append(score_of_choice - score_of_correct)
#             print(qid, file=fo)
#             print(q, file=fo)
#             print('%.3f\tA%s\t%s' % (scores[0], '*' if r=='A' else '', aa), file=fo)
#             print('%.3f\tB%s\t%s' % (scores[1], '*' if r=='B' else '', ab), file=fo)
#             print('%.3f\tC%s\t%s' % (scores[2], '*' if r=='C' else '', ac), file=fo)
#             print('%.3f\tD%s\t%s' % (scores[3], '*' if r=='D' else '', ad), file=fo)
#             print(file=fo)

print('accuracy     \t', np.mean(tries))
print('mean rank    \t', np.mean(ranks))
print('correct score\t', np.mean(scores_correct))
print('choice score \t', np.mean(scores_choice))
print('diff         \t', np.mean(scores_diff))

accuracy     	 0.2336
mean rank    	 1.1948
correct score	 0.432175727958
choice score 	 0.510042401325
diff         	 0.0778666733673


In [410]:
sentences = SentencesMorphy('/home/marat/ck12_tokens.txt')
qt = sentences.proc_line
# Q = 'When light passes through the lens of your eye, it produces __________ image on the retina.'
# A = 'a smaller'
# B = 'a larger'
# C = 'an upright'
# D = 'an inverse'

# Q = 'When light passes through the lens of your eye, .'
# A = 'it produces __________ image on the retina a smaller'
# B = 'it produces __________ image on the retina a larger'
# C = 'it produces __________ image on the retina an upright'
# D = 'it produces __________ image on the retina an inverse'

Q = 'When light passes through the lens of your eye, .'
A = 'it produces a smaller image on the retina'
B = 'it produces a larger image on the retina'
C = 'it produces an upright image on the retina'
D = 'it produces an inverse image on the retina'


print(qt(Q))
print(qt(A))
print(qt(B))
print(qt(C))
print(qt(D))
print(word_model.n_similarity(qt(Q), qt(A)))
print(word_model.n_similarity(qt(Q), qt(B)))
print(word_model.n_similarity(qt(Q), qt(C)))
print(word_model.n_similarity(qt(Q), qt(D)))

[u'light', u'pass', u'len', u'eye']
[u'produc', u'smaller', u'imag', u'retina']
[u'produc', u'larger', u'imag', u'retina']
[u'produc', u'upright', u'imag', u'retina']
[u'produc', u'invers', u'imag', u'retina']
0.623608499144
0.610877458023
0.595458377665
0.56252376595


0.16162788971417988

In [ ]:
#word_model.save("/data/datasets/a2_w2v.model")

In [466]:
def quantify_text(txt):
    ws = sentences.proc_line(txt)
    return [w for w in ws if w in word_model.vocab]


def range_answers(q, answers):
    question = quantify_text(q)
    ps = Counter()
    scores = []
    for a in answers:
        a_q = quantify_text(a)
        if a_q != []:
            si = word_model.n_similarity(question, a_q)
            scores.append(si)
        else:
            scores.append(0)
    return scores


def range_answers_mean_after_cosine(q, answers):
    question = quantify_text(q)
    ps = Counter()
    scores = []
    for a in answers:
        a_q = quantify_text(a)
        if a_q != []:
            coss = []
            for wq in question:
                word_sim = word_model.n_similarity([wq], a_q) 
                coss.append(word_sim * np.linalg.norm(word_model[wq]))  # **2 is under consideration
            scores.append(np.mean(coss))
        else:
            scores.append(0)
    return scores


def range_answers_best_question_word_after_cosine(q, answers):
    question = quantify_text(q)
    ps = Counter()
    scores = []
    for a in answers:
        a_q = quantify_text(a)
        if a_q != []:
            coss = []
            for wq in question:
                word_sim = word_model.n_similarity([wq], a_q) 
                coss.append(word_sim)
            scores.append(np.max(coss))
        else:
            scores.append(0)
    return scores


def range_answers_best_answer_word_after_cosine(q, answers):
    question = quantify_text(q)
    ps = Counter()
    scores = []
    for a in answers:
        a_q = quantify_text(a)
        if a_q != []:
            coss = []
            for wa in a_q:
                word_sim = word_model.n_similarity([wa], question) 
                coss.append(word_sim)
            scores.append(np.max(coss))
        else:
            scores.append(0)
    return scores

In [471]:
TRAIN_SET = '/home/marat/Downloads/training_set.tsv'
TRAIN_SET_ANSWERS = '/home/marat/allen-training-answers.tsv'
tries = []
ranks = []
scores_correct = []
scores_choice = []
scores_diff = []
with open(TRAIN_SET) as f:
    with open(TRAIN_SET_ANSWERS, 'w') as fo:
        next(f)
        for i, l in enumerate(f):
            [qid, q, r, aa, ab, ac, ad] = l.strip().split("\t")
            scores1 = np.array(range_answers(q, [aa, ab, ac, ad]))
            scores2 = np.array(range_answers_mean_after_cosine(q, [aa, ab, ac, ad]))
            scores = list(scores1/(1e-10 + np.linalg.norm(scores1)) + scores2/(1e-10+np.linalg.norm(scores2)))
#             print(scores)
            
            score_of_correct = scores['ABCD'.index(r)]
            score_of_choice = max(scores)
            guess = "ABCD"[scores.index(score_of_choice)]
            tries.append(1 if guess == r else 0)
            ranks.append(sorted(scores, reverse=True).index(score_of_correct))
            scores_choice.append(score_of_choice)
            scores_correct.append(score_of_correct)
            scores_diff.append(score_of_choice - score_of_correct)
            print(qid, file=fo)
            print(q, file=fo)
            print('%.3f\tA%s\t%s' % (scores[0], '*' if r=='A' else '', aa), file=fo)
            print('%.3f\tB%s\t%s' % (scores[1], '*' if r=='B' else '', ab), file=fo)
            print('%.3f\tC%s\t%s' % (scores[2], '*' if r=='C' else '', ac), file=fo)
            print('%.3f\tD%s\t%s' % (scores[3], '*' if r=='D' else '', ad), file=fo)
            print(file=fo)

In [472]:
print('accuracy     \t', np.mean(tries))
print('mean rank    \t', np.mean(ranks))
print('correct score\t', np.mean(scores_correct))
print('choice score \t', np.mean(scores_choice))
print('diff         \t', np.mean(scores_diff))

accuracy     	 0.38
mean rank    	 1.1444
correct score	 1.05619328374
choice score 	 1.30864033155
diff         	 0.252447047812


In [473]:
len(tries)

2500

In [ ]:
tries[0:10]

In [ ]:
len(word_model.vocab)

In [22]:
with codecs.open(WORD2VEC_EXPORT, "w", encoding='utf8') as fo:
    for w in word_model.vocab:
        fo.write("%s %s\n" % (w, " ".join(map(str, word_model[w]))))

In [305]:
VALIDATION_SET = "/home/marat/Downloads/validation_set.tsv"
SUBMISSION = "/home/marat/allen-ai-submission.csv"

In [348]:
with open(SUBMISSION, "w") as s:
    s.write("id,correctAnswer\n")
    with codecs.open(VALIDATION_SET, encoding='utf8') as fv:
        next(fv)
        for l in fv:
            [qid, q, aa, ab, ac, ad] = l.strip().split("\t")
            scores = range_answers(q, [aa, ab, ac, ad])
            guess = "ABCD"[scores.index(max(scores))]
            s.write("%s,%s\n" % (qid, guess))

In [491]:
from sklearn.metrics.pairwise import cosine_similarity

r='''100004
Which is a distinction between an epidemic and a pandemic?
B
the symptoms of the disease
the geographical area affected
the species of organisms infected
the season in which the disease spreads'''
I, Q, correct, A, B, C, D = r.strip().split('\n')

def diff(vec, words):
    return cosine_similarity(vec, np.array([word_model[w] for w in words]).sum(axis=0))

d = - word_model[qt('epidemic')[0]] + word_model[qt('pandemic')[0]]
print(diff(d, qt(A)))
print(diff(d, qt(B)))
print(diff(d, qt(C)))
print(diff(d, qt(D)))

    

# print(word_model.n_similarity(qt('epidemic') - qt('pandemic'), qt(A)))
# print(word_model.n_similarity(qt('epidemic') - qt('pandemic'), qt(B)))
# print(word_model.n_similarity(qt('epidemic') - qt('pandemic'), qt(C)))
# print(word_model.n_similarity(qt('epidemic') - qt('pandemic'), qt(D)))

[[-0.05650566]]
[[-0.06143067]]
[[-0.03365701]]
[[-0.13350873]]
